##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [3]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

## Predict Shakespeare with Cloud TPUs and Keras

## Overview

This example uses [tf.keras](https://www.tensorflow.org/guide/keras) to build a *language model* and train it on a Cloud TPU. This language model predicts the next character of text given the text so far. The trained model can generate new snippets of text that read in a similar style to the text training data.

The model trains for 10 epochs and completes in approximately 5 minutes.

This notebook is hosted on GitHub. To view it in its original repository, after opening the notebook, select **File > View on GitHub**.

## Learning objectives

In this Colab, you will learn how to:
*   Build a two-layer, forward-LSTM model.
*   Use distribution strategy to produce a `tf.keras` model that runs on TPU version and then use the standard Keras methods to train: `fit`, `predict`, and `evaluate`.
*   Use the trained model to make predictions and generate your own Shakespeare-esque play.






## Instructions

<h3>  &nbsp;&nbsp;Train on TPU&nbsp;&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a></h3>

   1. On the main menu, click Runtime and select **Change runtime type**. Set "TPU" as the hardware accelerator.
   1. Click Runtime again and select **Runtime > Run All**. You can also run the cells manually with Shift-ENTER.

TPUs are located in Google Cloud, for optimal performance, they read data directly from Google Cloud Storage (GCS)

## Data, model, and training

In this example, you train the model on the combined works of William Shakespeare, then use the model to compose a play in the style of *The Great Bard*:

<blockquote>
Loves that led me no dumbs lack her Berjoy's face with her to-day.  
The spirits roar'd; which shames which within his powers  
	Which tied up remedies lending with occasion,  
A loud and Lancaster, stabb'd in me  
	Upon my sword for ever: 'Agripo'er, his days let me free.  
	Stop it of that word, be so: at Lear,  
	When I did profess the hour-stranger for my life,  
	When I did sink to be cried how for aught;  
	Some beds which seeks chaste senses prove burning;  
But he perforces seen in her eyes so fast;  
And _  
</blockquote>


### Download data

Download *The Complete Works of William Shakespeare* as a single text file from [Project Gutenberg](https://www.gutenberg.org/). You use snippets from this file as the *training data* for the model. The *target* snippet is offset by one character.

In [3]:
!wget --show-progress --continue -O /content/shakespeare.txt http://www.gutenberg.org/files/100/100-0.txt

--2025-09-21 19:16:49--  http://www.gutenberg.org/files/100/100-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.gutenberg.org/files/100/100-0.txt [following]
--2025-09-21 19:16:49--  https://www.gutenberg.org/files/100/100-0.txt
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5422721 (5.2M) [text/plain]
Saving to: ‘/content/shakespeare.txt’

/content/shakespear 100%[===================>]   5.17M  5.94MB/s    in 0.9s    

2025-09-21 19:16:51 (5.94 MB/s) - ‘/content/shakespeare.txt’ saved [5422721/5422721]



### Build the input dataset

We just downloaded some text. The following shows the start of the text and a random snippet so we can get a feel for the whole text.

In [4]:
!head -n5 /content/shakespeare.txt
!echo "..."
!shuf -n5 /content/shakespeare.txt

*** START OF THE PROJECT GUTENBERG EBOOK 100 ***




...
HERMIA.
Whither, my lord? Cesario, husband, stay.
   And bowl the round nave down the hill of heaven,

 Enter a Herald.


In [5]:
import numpy as np
import tensorflow as tf
import os

from packaging.version import parse

if parse(tf.__version__) < parse('2.0'):
    raise Exception('This notebook is compatible with TensorFlow 2.0 or higher.')

SHAKESPEARE_TXT = '/content/shakespeare.txt'

def transform(txt):
  return np.asarray([ord(c) for c in txt if ord(c) < 255], dtype=np.int32)

def input_fn(seq_len=100, batch_size=1024):
  """Return a dataset of source and target sequences for training."""
  with tf.io.gfile.GFile(SHAKESPEARE_TXT, 'r') as f:
    txt = f.read()

  source = tf.constant(transform(txt), dtype=tf.int32)

  ds = tf.data.Dataset.from_tensor_slices(source).batch(seq_len+1, drop_remainder=True)

  def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

  BUFFER_SIZE = 10000
  ds = ds.map(split_input_target).shuffle(BUFFER_SIZE).batch(batch_size, drop_remainder=True)

  return ds.repeat()

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:82: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


In [1]:
!pip install packaging

In [2]:
!pip install tensorflow numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 113.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 117.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 159.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.6 MB/s eta 0:00:00


### Build the model

The model is defined as a two-layer, forward-LSTM, the same model should work both on CPU and TPU.

Because our vocabulary size is 256, the input dimension to the Embedding layer is 256.

When specifying the arguments to the LSTM, it is important to note how the stateful argument is used. When training we will make sure that `stateful=False` because we do want to reset the state of our model between batches, but when sampling (computing predictions) from a trained model, we want `stateful=True` so that the model can retain information across the current batch and generate more interesting text.

In [6]:
EMBEDDING_DIM = 512

def lstm_model(seq_len=100, batch_size=None, stateful=True):
  """Language model: predict the next word given the current word."""
  source = tf.keras.Input(
      name='seed', shape=(seq_len,), batch_size=batch_size, dtype=tf.int32)

  embedding = tf.keras.layers.Embedding(input_dim=256, output_dim=EMBEDDING_DIM)(source)
  lstm_1 = tf.keras.layers.LSTM(EMBEDDING_DIM, stateful=stateful, return_sequences=True)(embedding)
  lstm_2 = tf.keras.layers.LSTM(EMBEDDING_DIM, stateful=stateful, return_sequences=True)(lstm_1)
  predicted_char = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256, activation='softmax'))(lstm_2)
  return tf.keras.Model(inputs=[source], outputs=[predicted_char])

### Train the model

First, we need to create a distribution strategy that can use the TPU. In this case it is TPUStrategy. You can create and compile the model inside its scope. Once that is done, future calls to the standard Keras methods `fit`, `evaluate` and `predict` use the TPU.

Again note that we train with `stateful=False` because while training, we only care about one batch at a time.

In [7]:
import tensorflow as tf
import os

try:
    # This is a more direct way to initialize the TPU system.
    # It can sometimes work when the standard .connect() method fails.
    tf.tpu.experimental.initialize_tpu_system()

    # After initialization, we find the resolver.
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

    # Now, create the strategy with the found resolver.
    strategy = tf.distribute.TPUStrategy(resolver)

    print("✅✅ Success! TPU connected using the direct initialization method.")
    print("All devices: ", tf.config.list_logical_devices('TPU'))

except Exception as e:
    print(f"❌ Final attempt failed: {e}")

❌ Final attempt failed: Please provide a TPU Name to connect to.


In [ ]:
import tensorflow as tf

# --- CORRECTED TPU CONNECTION ---
# This is the modern and correct way to connect to any Colab TPU.
# It automatically finds the TPU assigned to your session.
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    strategy = tf.distribute.TPUStrategy(resolver)
    print("✅ TPU connected successfully!")
except Exception as e:
    print(f"❌ TPU connection failed: {e}")
    # If it fails, fall back to the default strategy to run on CPU/GPU
    strategy = tf.distribute.get_strategy()
    print("⚠️ Running with default strategy (CPU/GPU).")

# --- YOUR MODEL TRAINING CODE ---
# Now, run your training code within the strategy scope.
# This ensures the model is built and trained on the TPU.
with strategy.scope():
    # Make sure lstm_model and input_fn are defined in your notebook
    training_model = lstm_model(seq_len=100, stateful=False)
    training_model.compile(
        optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.01),
        loss='sparse_categorical_crossentropy',
        metrics=['sparse_categorical_accuracy'])

print("\nStarting model training on the selected device...")
training_model.fit(
    input_fn(),
    steps_per_epoch=100,
    epochs=35
)

# Save the weights after training
training_model.save_weights('/tmp/bard.h5', overwrite=True)
print("\n✅ Training complete and weights saved to /tmp/bard.h5")

❌ TPU connection failed: Please provide a TPU Name to connect to.
⚠️ Running with default strategy (CPU/GPU).

Starting model training on the selected device...
Epoch 1/35
100/100 ━━━━━━━━━━━━━━━━━━━━ 302s 3s/step - loss: 3.9323 - sparse_categorical_accuracy: 0.1132
Epoch 2/35
100/100 ━━━━━━━━━━━━━━━━━━━━ 298s 3s/step - loss: 3.1250 - sparse_categorical_accuracy: 0.1502
Epoch 3/35
100/100 ━━━━━━━━━━━━━━━━━━━━ 297s 3s/step - loss: 2.8991 - sparse_categorical_accuracy: 0.1757
Epoch 4/35
100/100 ━━━━━━━━━━━━━━━━━━━━ 297s 3s/step - loss: 2.6703 - sparse_categorical_accuracy: 0.2340
Epoch 5/35
100/100 ━━━━━━━━━━━━━━━━━━━━ 297s 3s/step - loss: 2.5200 - sparse_categorical_accuracy: 0.2753
Epoch 6/35
100/100 ━━━━━━━━━━━━━━━━━━━━ 297s 3s/step - loss: 2.3960 - sparse_categorical_accuracy: 0.3020
Epoch 7/35
100/100 ━━━━━━━━━━━━━━━━━━━━ 297s 3s/step - loss: 2.3291 - sparse_categorical_accuracy: 0.3223
Epoch 8/35
 96/100 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - loss: 2.2657 - sparse_categorical_accuracy:

### Make predictions with the model

Use the trained model to make predictions and generate your own Shakespeare-esque play.
Start the model off with a *seed* sentence, then generate 250 characters from it. The model makes five predictions from the initial seed.

The predictions are done on the CPU so the batch size (5) in this case does not have to be divisible by 8.

Note that when we are doing predictions or, to be more precise, text generation, we set `stateful=True` so that the model's state is kept between batches. If stateful is false, the model state is reset between each batch, and the model will only be able to use the information from the current batch (a single character) to make a prediction.

The output of the model is a set of probabilities for the next character (given the input so far). To build a paragraph, we predict one character at a time and sample a character (based on the probabilities provided by the model). For example, if the input character is "o" and the output probabilities are "p" (0.65), "t" (0.30), others characters (0.05), then we allow our model to generate text other than just "Ophelia" and "Othello."

In [ ]:
BATCH_SIZE = 5
PREDICT_LEN = 250

# Keras requires the batch size be specified ahead of time for stateful models.
# We use a sequence length of 1, as we will be feeding in one character at a
# time and predicting the next character.
prediction_model = lstm_model(seq_len=1, batch_size=BATCH_SIZE, stateful=True)
prediction_model.load_weights('/tmp/bard.h5')

# We seed the model with our initial string, copied BATCH_SIZE times

seed_txt = 'Looks it not like the king?  Verily, we must go! '
seed = transform(seed_txt)
seed = np.repeat(np.expand_dims(seed, 0), BATCH_SIZE, axis=0)

# First, run the seed forward to prime the state of the model.
prediction_model.reset_states()
for i in range(len(seed_txt) - 1):
  prediction_model.predict(seed[:, i:i + 1])

# Now we can accumulate predictions!
predictions = [seed[:, -1:]]
for i in range(PREDICT_LEN):
  last_word = predictions[-1]
  next_probits = prediction_model.predict(last_word)[:, 0, :]

  # sample from our output distribution
  next_idx = [
      np.random.choice(256, p=next_probits[i])
      for i in range(BATCH_SIZE)
  ]
  predictions.append(np.asarray(next_idx, dtype=np.int32))


for i in range(BATCH_SIZE):
  print('PREDICTION %d\n\n' % i)
  p = [predictions[j][i] for j in range(PREDICT_LEN)]
  generated = ''.join([chr(c) for c in p])  # Convert back to text
  print(generated)
  print()
  assert len(generated) == PREDICT_LEN, 'Generated text too short'

## What's next

* Learn about [Cloud TPUs](https://cloud.google.com/tpu/docs) that Google designed and optimized specifically to speed up and scale up ML workloads for training and inference and to enable ML engineers and researchers to iterate more quickly.
* Explore the range of [Cloud TPU tutorials and Colabs](https://cloud.google.com/tpu/docs/tutorials) to find other examples that can be used when implementing your ML project.

On Google Cloud Platform, in addition to GPUs and TPUs available on pre-configured [deep learning VMs](https://cloud.google.com/deep-learning-vm/),  you will find [AutoML](https://cloud.google.com/automl/)*(beta)* for training custom models without writing code and [Cloud ML Engine](https://cloud.google.com/ml-engine/docs/) which will allows you to run parallel trainings and hyperparameter tuning of your custom models on powerful distributed hardware.
